In [ ]:
import numpy
import pandas
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor
from sklearn import tree
from sklearn.model_selection import train_test_split

housing=pandas.read_csv('C:/Users/000110888/OneDrive - CSULB/Desktop/housing_data.csv')
coding={'<1H OCEAN': 1, 'INLAND': 2, 'NEAR BAY': 3, 'NEAR OCEAN': 4}
housing['ocean_proximity']=housing['ocean_proximity'].map(coding)
X=housing.iloc[:,0:7].values
y=housing.iloc[:,7].values

#SPLITTING DATA INTO 80% TRAINING AND 20% TESTING SETS
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.20, 
random_state=348644)

#FITTING REGRESSION TREE WITH RSS SPLITTING CRITERION
rtree = DecisionTreeRegressor(random_state=907420, 
criterion="squared_error", max_leaf_nodes=5)
reg_tree_RSS = rtree.fit(X_train, y_train)

#PLOTTING FITTED TREE
fig=plt.figure(figsize=(15,10))
fn=['housing_median_age','total_rooms','total_bedrooms','population',
'households','median_income','ocean_proximity']
tree.plot_tree(reg_tree_RSS, feature_names=fn, filled=True)

#COMPUTING PREDICTION ACCURACY FOR TESTING DATA
y_pred=reg_tree_RSS.predict(X_test)

ind10=[]
ind15=[]
ind20=[]     

for sub1, sub2 in zip(y_pred, y_test):
    ind10.append(1) if abs(sub1-sub2)<0.10*sub2 else ind10.append(0)
    ind15.append(1) if abs(sub1-sub2)<0.15*sub2 else ind15.append(0)
    ind20.append(1) if abs(sub1-sub2)<0.20*sub2 else ind20.append(0)
 
#accuracy within 10%
accuracy10=sum(ind10)/len(ind10)
print(accuracy10)

#accuracy within 15%
accuracy15=sum(ind15)/len(ind15)
print(accuracy15)

#accuracy within 20%
accuracy20=sum(ind20)/len(ind20)
print(accuracy20)

In [2]:
############################################################################

#FITTING REGRESSION TREE WITH CHAID SPLITTING CRITERION

#SPLITTING RESPONSE VARIABLE INTO DECILES AND MAKING IT NOMINAL
housing=pandas.read_csv('C:/Users/000110888/OneDrive - CSULB/Desktop/housing_data.csv')
coding={'<1H OCEAN': 1, 'INLAND': 2, 'NEAR BAY': 3, 'NEAR OCEAN': 4}
housing['ocean_proximity']=housing['ocean_proximity'].map(coding)

housing['deciles']=pandas.qcut(housing['median_house_value'], 10, labels=False)
deciles_coding={0:'0th',1:'1st',2:'2nd',3:'3rd',4:'4th',5:'5th',6:'6th',7:'7th',8:'8th',9:'9th'}
housing['deciles']=housing['deciles'].map(deciles_coding)

X=housing.iloc[:,0:7].values
y=housing.iloc[:,7:9].values

#SPLITTING DATA INTO 80% TRAINING AND 20% TESTING SETS
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.20, random_state=348644)

X_train=pandas.DataFrame(X_train, columns=['housing_median_age','total_rooms',
'total_bedrooms','population','households','median_income','ocean_proximity'])
y_train_cat=pandas.DataFrame(y_train[:,1], columns=['deciles'])
train_data=pandas.concat([X_train, y_train_cat],axis=1)

#FITTING TREE
!pip install chefboost
from chefboost import Chefboost

config={'algorithm': 'CHAID', 'max_depth': 4}
tree_chaid=Chefboost.fit(train_data, config, target_label='deciles')

#COMPUTING PREDICTION ACCURACY FOR TESTING DATA
y_pred=[]
for i in range(len(y_test)):
    y_pred.append(Chefboost.predict(tree_chaid, X_test[i,:]))
    
y_test_act=pandas.DataFrame(y_test[:,0], columns=['median_house_value'])
y_pred_class=pandas.DataFrame(y_pred, columns=['deciles'])

pred_data=pandas.concat([y_test_act,y_pred_class],axis=1)

y_train_all=pandas.DataFrame(y_train[:,:], columns=['median_house_value','deciles'])
pred_value=y_train_all.groupby('deciles')['median_house_value'].mean() #computes means for each decile

inner_join = pandas.merge(pred_data, pred_value, on='deciles', how ='inner')

ind10=[]
ind15=[]
ind20=[]     
#median_house_value_x=observed value, median_house_value_y=predicted value
for sub1, sub2 in zip(inner_join['median_house_value_x'], inner_join['median_house_value_y']):
    ind10.append(1) if abs(sub1-sub2)<0.10*sub1 else ind10.append(0)
    ind15.append(1) if abs(sub1-sub2)<0.15*sub1 else ind15.append(0)
    ind20.append(1) if abs(sub1-sub2)<0.20*sub1 else ind20.append(0)
 
#accuracy within 10%
accuracy10=sum(ind10)/len(ind10)
print(accuracy10)

#accuracy within 15%
accuracy15=sum(ind15)/len(ind15)
print(accuracy15)

#accuracy within 20%
accuracy20=sum(ind20)/len(ind20)
print(accuracy20)


26-02-04 16:20:00 - CHAID tree is going to be built...
26-02-04 16:20:07 - -------------------------
26-02-04 16:20:07 - finished in 7.314399719238281 seconds
26-02-04 16:20:07 - -------------------------
26-02-04 16:20:07 - Evaluate train set
26-02-04 16:20:07 - -------------------------
26-02-04 16:20:07 - Accuracy: 26.484821821381434% on 2273 instances
26-02-04 16:20:07 - Labels: ['2nd' '1st' '0th' '5th' '6th' '9th' '7th' '3rd' '4th' '8th']
26-02-04 16:20:07 - Confusion matrix: [[17, 4, 0, 16, 8, 12, 15, 9, 6, 14], [27, 66, 62, 3, 2, 1, 2, 25, 13, 1], [56, 89, 138, 15, 5, 2, 0, 52, 48, 3], [39, 27, 5, 57, 25, 2, 21, 34, 20, 8], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2, 1, 0, 35, 53, 118, 65, 5, 12, 82], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [63, 49, 18, 62, 42, 6, 35, 87, 79, 16], [13, 1, 2, 22, 24, 26, 7, 14, 31, 11], [10, 1, 0, 28, 58, 65, 68, 4, 21, 88]]
26-02-04 16:20:07 - Decision 2nd
26-02-04 16:20:07 - Accuracy: 87.0656
26-02-04 16:20:07 - Precision: 16.8317%, Recall: 7.489%, F1: 10.3659